In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn.decomposition as sk
import Kernel_PCA as FP 
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import PCA


In [2]:
Trial_1 = FP.clean_data("Trial1")
Trial_2 = FP.clean_data("Trial2")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Trial1'

In [ ]:
#### We are going to average 10 signals here. Uncomment 
for i in range(0,10):
    c4_c3 = []
    gaussian_noise = 0 #np.random.normal(0,.001,Trial_1[i][0].shape)
    
    ## Choose random data in the trials
    c4_c3.append(Trial_1[np.random.randint(0,83,1)[0]][1]+gaussian_noise)

In [ ]:
#### Compute KPCA gaussian

### I chose the only parameter for the gaussian kernel to be sigma=10
temp = []
sigma = 10
eigenvalues,eigenvectors = np.linalg.eig(FP.Gram_Matrix(FP.K(c4_c3,sigma)))

for z in c4_c3:
    p = FP.inverse_function(z,10,eigenvalues,eigenvectors,c4_c3,sigma)
    temp.append(p)
    


In [ ]:
#### Calculate mean of K
temp = np.array(temp)
trials = len(c4_c3)
average = np.mean(temp,axis=0)
x = np.arange(0,len(average))/6.4
plt.xlabel("milliseconds")
plt.ylabel("Mv")
plt.plot(x,np.transpose(temp))
#plt.plot(np.transpose(c4_c3))
plt.title("SSEP signal using KPCA before averaging")
plt.show()


In [ ]:
#### Compute PCA
pca = FP.PCA_(c4_c3)

#### Calculate mean of PCA
p = np.mean(temp,axis=0)
pca_ = np.transpose(pca)
x = np.arange(0,len(p))/6.4
plt.xlabel("milliseconds")
plt.ylabel("Mv")
plt.plot(x,pca_)
plt.title("SSEP signal using PCA before averaging")
plt.savefig("c4_c3 SSEP KPCA")

In [ ]:
### walsch convolution
gauss = FP.walsch_convolution(average)
pca_walsch = FP.walsch_convolution(np.mean(pca,axis=0))

## Moving average
gauss = FP.clip(np.convolve([1,1,1],gauss),100,630)
pca_walsch = np.convolve([1,1,1],pca_walsch)
plt.plot(FP.clip(pca_walsch,100,630),label="KPCA walsch")
plt.plot(FP.clip(gauss,100,630),label="PCA walsch")
plt.legend()
plt.show()
plt.savefig("c4_c3 walsch ")

In [ ]:
### Clip ends of vector
gauss = FP.clip(gauss,200,350)
pca = FP.clip(pca_walsch,200,350)

gauss_max = np.argmax(gauss)
gauss_min = np.argmin(gauss)

pca_max = np.argmax(pca)
pca_min = np.argmin(pca)

In [ ]:
print("N45 Time, gaussian kernel",FP.percent_time_difference(gauss_max/6.4,42.7))
print("P37 time, gaussian kernel",FP.percent_time_difference(gauss_min/6.4,36.9))

print("N45 Time, PCA",FP.percent_time_difference(pca_max/6.4,42.7))
print("P37 time, PCA",FP.percent_time_difference(pca_min/6.4,36.9))



In [ ]:
clinical_mean = np.mean(c4_c3,axis=0)
clinical_max = np.argmax(FP.clip(FP.walsch_convolution(np.mean(c4_c3,axis=0)),200,350))
clinical_min = np.argmin(FP.clip(FP.walsch_convolution(np.mean(c4_c3,axis=0)),200,350))

In [ ]:
np.abs(FP.peak_to_peak_percentage(average[gauss_max]-average[gauss_min],clinical_mean[clinical_max]-clinical_mean[clinical_min]))


In [ ]:
np.abs(FP.peak_to_peak_percentage(p[gauss_max]-p[gauss_min],clinical_mean[clinical_max]-clinical_mean[clinical_min]))